In [1]:
import os 
os.environ['HUGGINGFACE_API_KEY'] = "api-key"

In [2]:
import pandas as pd
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig, 
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [3]:
dataset = load_dataset("dhruvvaidh/cover-letter-dataset-llama3")

Generating train split:   0%|          | 0/813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/349 [00:00<?, ? examples/s]

In [6]:
base_model_name = "meta-llama/Llama-3.2-1B-Instruct"
new_model_name = "dhruvvaidh/cover-letter-gen-llama2"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.save_pretrained(save_directory="./tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/tokenizer.json')

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True # We are quantizing the quantized paramters to save more space
)

peft_config = LoraConfig(
    lora_alpha = 32,#Strength of the Adapters or their impact on the model as adapter weights get fused with a small subset of params in og model
    lora_dropout = 0.05,
    r = 16,
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                            quantization_config = bnb_config)

kbit_model = prepare_model_for_kbit_training(
    model
)

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 4,
    per_device_train_batch_size = 10,
    gradient_accumulation_steps = 1,
    eval_steps = 1000,
    logging_steps = 1,
    optim = "paged_adamw_8bit",
    learning_rate = 2e-4,
    lr_scheduler_type = "linear",
    warmup_steps = 10,
    report_to = "wandb",
)

trainer = SFTTrainer(
    model = kbit_model,
    train_dataset = dataset['train'],
    eval_dataset = dataset['test'],
    peft_config = peft_config,
    dataset_text_field = "Prompt",
    max_seq_length=512,
    tokenizer = tokenizer,
    args = training_arguments
)

trainer.train()

In [ ]:
trainer.model.push_to_hub(new_model_name,token='api-key')